In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft,ifft
import pandas as pd
from scipy.optimize import minimize
from pprint import pprint
from iminuit import Minuit

In [2]:
# N=20取bin的函数

def databin_20(lst):
    result = [[]]
    length = len(lst)
    n = 0
    for i in range(length):
        result[-1].append(lst[i])
        n = n+1
        if n == 20:
            n = 0
            result.append([])
    output=[]
    for j in range(len(result)):
        output.append(np.mean(result[j])) 
    return output

def databin_20_std(lst):
    result = [[]]
    length = len(lst)
    n = 0
    for i in range(length):
        result[-1].append(lst[i])
        n = n+1
        if n == 20:
            n = 0
            result.append([])        
    output=[]
    for j in range(len(result)):
        output.append(np.std(result[j])) 
    return output


In [3]:
def chi2_of_model(x0):
    A,f_b,alpha_H=x0
    
    data = pd.read_csv("0.3_10_final.csv")  
    counts_data = data['RATE']
    dt=200.
    mean_x_data = np.mean(counts_data)
    N=len(counts_data)
    N_randomlc=len(counts_data)
    
    omega = []
    POW = []
    DFT = []
    fr = []
    fi = []
    f1 = []
    f2 = []
    p = []
    f = []
    
    counts_list=[None for i in range(500)]
    
    
    # 500条光变曲线
    for a in range(500): 
        alpha_L=1.0
        for j in range(1,int(N_randomlc)+1):
            omega.append(j/(N_randomlc*dt))
            POW.append(((omega[-1]**(-alpha_L))/(1.0+(omega[-1]/f_b)**(alpha_H-alpha_L)))*A)
            DFT.append(complex(np.sqrt(POW[-1]),np.sqrt(POW[-1])))
            s1=np.random.normal(loc=0.0, scale=1.0, size=None)
            s2=np.random.normal(loc=0.0, scale=1.0, size=None)
            fr.append((DFT[-1].real)*s1)
            fi.append((DFT[-1].imag)*s2)
            f1.append(complex(fr[-1],fi[-1]))
        counts = ifft(f1)
        counts_list[a]=counts
    
    
    #周期图
    per_list=[None for i in range(500)]
    for a in range(500):
        nf = N/2.0 
        df = 1.0/(dt*N)
        F_a = np.arange(1,nf+1)
        F = [i*df for i in F_a]
        F1 = F[0:int(nf)]
        mean_x = np.mean(counts_list[a])
        x_new  = [i-mean_x for i in counts_list[a]]
        dft   = fft(counts_list[a])
        dft_1 = dft[1:int(nf)+1]
        per = (abs(dft_1)**2)
        per_list[a] = per
    
    
    # 500条周期图取 mean,std
    per_everypoint_list=[]
    for i in range(int(nf)):
        per_everypoint_list.append([])
    ADM=[]
    ADM_std=[]
    for m in range(int(nf)):
        for n in range(500):
            per_everypoint_list[m].append(per_list[n][m])
        
    for m in range(int(nf)):
        ADM.append(np.mean(per_everypoint_list[m]))
        ADM_std.append(np.std(per_everypoint_list[m]))    
    

    ADM_times_f = np.multiply(np.array(F),np.array(ADM))
        
        
    POW1=POW[0:int(nf)]
    P_TIMES_F = np.multiply(np.array(F),np.array(POW1))
    
    ADM_with_poisson = [i+2/mean_x_data for i in ADM]
    ADM_times_f_ps = np.multiply(np.array(F),np.array(ADM_with_poisson))
    
    
    #加上泊松项并分bin
    ADM_binned_ps = databin_20(ADM_with_poisson)
    F1_binned = databin_20(F1)
    ADM_times_f_b_ps = np.multiply(np.array(F1_binned),np.array(ADM_binned_ps))
    
    
    # 从数据得到参考周期图
    pnum = np.arange(len(counts_data))
    t = [i*dt for i in pnum]
    
    nf = N/2.0
    df = 1.0/(dt*N)
    F_num = np.arange(1,nf)
    F = [i*df for i in F_num]
    mean_x = np.mean(counts_data)
    dft = fft(counts_data)
    dft1= dft[1:int(nf)]
    per_data = (abs(dft1)**2.0)*2.0*dt/((mean_x**2.0)*N)
    p_times_f = np.multiply(np.array(F),np.array(per_data))
    
    
    # 数据分bin
    per_data_binned = databin_20(per_data)
    per_data_b_std = databin_20_std(per_data)
    p_times_f_data_b = np.multiply(np.array(F1_binned),np.array(per_data_binned))
    
    
    chi2=0
    for i in range(len(ADM_binned_ps)):
        chi2 = chi2+(((ADM_binned_ps[i]-per_data_binned[i])/per_data_b_std[i])**2)    
    
    print('chi2 of this time = ',chi2)
    
    return (chi2)

In [4]:
x0=[0.005,1.7E-4,3.8]
x0_array=np.array(x0)
m = minimize(chi2_of_model,x0_array,method='TNC')

chi2 of this time =  11.77163438950981
chi2 of this time =  9.493373005693448
chi2 of this time =  10.528896941938296
chi2 of this time =  9.330680106527197
chi2 of this time =  10.46544937864194
chi2 of this time =  8.054686479951949
chi2 of this time =  16.1979436005815
chi2 of this time =  13.447137116493565
chi2 of this time =  8.42949033953713
chi2 of this time =  9.619997252797516
chi2 of this time =  18.27698362913024
chi2 of this time =  9.295149319781034
chi2 of this time =  11.824207563690138
chi2 of this time =  8.164910393641527
chi2 of this time =  23.8518198880153
chi2 of this time =  21.514458872139286
chi2 of this time =  11.505807557828218
chi2 of this time =  9.530983932410509
chi2 of this time =  9.510862862183682
chi2 of this time =  12.319203915109796
chi2 of this time =  9.231925475469502
chi2 of this time =  11.828249733888283
chi2 of this time =  8.676836091413218
chi2 of this time =  11.658953983036803
chi2 of this time =  24.803622804504794
chi2 of this time =

In [5]:
print(m)

     fun: 17.225553043372116
     jac: array([ 1.57857164e+08,  2.35327929e+08, -8.14994431e+07])
 message: 'Converged (|x_n-x_(n-1)| ~= 0)'
    nfev: 5
     nit: 1
  status: 2
 success: True
       x: array([4.99999989e-03, 1.70000017e-04, 3.80000000e+00])
